# Cachegrind 

Cachegrind simulates the first-level and last-level caches. The reason for this choice is that the last-level cache has the most influence on runtime, as it masks accesses to main memory. Furthermore, the L1 caches often have low associativity, so simulating them can detect cases where the code interacts badly with this cache (eg. traversing a matrix column-wise with the row length being a power of 2).

Therefore, Cachegrind always refers to the I1, D1 and LL (last-level) caches.

Cachegrind gathers the following statistics (abbreviations used for each statistic is given in parentheses):

- I cache reads (Ir, which equals the number of instructions executed), I1 cache read misses (I1mr) and LL cache instruction read misses (ILmr).

- D cache reads (Dr, which equals the number of memory reads), D1 cache read misses (D1mr), and LL cache data read misses (DLmr).

- D cache writes (Dw, which equals the number of memory writes), D1 cache write misses (D1mw), and LL cache data write misses (DLmw).

- Conditional branches executed (Bc) and conditional branches mispredicted (Bcm).

- Indirect branches executed (Bi) and indirect branches mispredicted (Bim).

Note that D1 total accesses is given by D1mr + D1mw, and that LL total accesses is given by ILmr + DLmr + DLmw.

On a modern machine, an L1 miss will typically cost around 10 cycles, an LL miss can cost as much as 200 cycles, and a mispredicted branch costs in the region of 10 to 30 cycles. 

LL miss rate is computed relative to the total number of memory accesses, not the number of L1 misses. I.e. it is (ILmr + DLmr + DLmw) / (Ir + Dr + Dw) not (ILmr + DLmr + DLmw) / (I1mr + D1mr + D1mw)

Ir : I cache reads (ie. instructions executed)
I1mr: I1 cache read misses
I2mr: L2 cache instruction read misses
Dr : D cache reads (ie. memory reads)
D1mr: D1 cache read misses
D2mr: L2 cache data read misses
Dw : D cache writes (ie. memory writes)
D1mw: D1 cache write misses
D2mw: L2 cache data write misses


In [1]:
import re
import numpy as np    
import pandas as pd

In [2]:
vertice = [100,1000 ,5000, 10000 ,20000]
edge = [100,500,1000,5000,10000, 20000, 40000, 80000]
# Orig, approxChol_opt, approxChol_vector2, approxChol_vector2_merge, approxChol_vector2_opt2, approxChol_vector2_mergerand
func_name = ['approxChol(LLMatOrd)', 'approxChol_opt(LLMatOrd)', 'approxChol_vector2(LLMatOrd_vector2)', 
             'approxChol_vector2_merge(LLMatOrd_vector2)', 'approxChol_vector2_opt2(LLMatOrd_vector2)', 
             'approxChol_vector2_mergerand(LLMatOrd_vector2)']
func_alias = ['Baseline', 'Inline', 'Vector', 'Merge', 'Merge+SIMD', 'Merge+Rand']

In [3]:
datalist = []
header=['#V','#E','Ir','I1mr','ILmr','Dr','D1mr','DLmr','Dw','D1mw','DLmw','Bc','Bcm','Bi','Bim','function']
with open("./cache.log", 'r') as f:
    next(f)
    for line in f:
        line = line.split()
        num = [int(x.replace(',', '').strip()) for x in line[:-1]]
        num.append(line[-1])
        datalist.append(num)
res = pd.DataFrame(datalist, columns=header)
res.convert_dtypes()
res.dtypes

#V           int64
#E           int64
Ir           int64
I1mr         int64
ILmr         int64
Dr           int64
D1mr         int64
DLmr         int64
Dw           int64
D1mw         int64
DLmw         int64
Bc           int64
Bcm          int64
Bi           int64
Bim          int64
function    object
dtype: object

In [4]:
datalist = []
header=['flops','flcomps', 'intops', 'intcomps', 'total', 'cycles','flops/cycle']
with open("./exact_cycle.txt", 'r') as f:
    for line in f:
        line = line.split()
        num = [int(x.replace(',', '').strip()) for x in line[2:-3]]
        num.append(float(line[-3]))
        num.append(float(line[-2]))
        datalist.append(num)
res2 = pd.DataFrame(datalist, columns=header)
res2 = res2.convert_dtypes()
res2

,flops,flcomps,intops,intcomps,total,cycles,flops/cycle
0,12156,7819,13649,36298,69922,389294,0.179612
1,12156,7819,13649,36298,69922,261295,0.267598
2,12156,7819,13649,36298,69922,365875,0.191109
3,12156,7819,13649,36298,69922,372043,0.187941
4,12156,7819,13649,36298,69922,344799,0.202791
...,...,...,...,...,...,...,...
79,4481753,3273707,5586980,14173088,27515528,2482130000,0.011085
80,4481753,3273707,5586980,14173088,27515528,243069000,0.113201
81,4481753,3273707,5586980,14173088,27515528,213610000,0.128812
82,4481753,3273707,5586980,14173088,27515528,198732000,0.138456


In [5]:
result = pd.concat([res,res2], axis=1)

In [6]:
result['LL miss rate'] = (result['ILmr'] + result['DLmr'] + result['DLmw']) / (result['Ir'] + result['Dr'] + result['Dw']) *100
result['mem'] = result['Dr'] + result['Dw']
result['branch miss rate'] = result['Bcm']/result['Bc']*100

In [7]:
func_alias=['Baseline', 'Inline','Vector', 'Merge', 'Merge+SIMD','Merge+Rand']

runtime = []
with open("./runtime.log", 'r') as f:
    next(f)
    for line in f:
        line = line.split()
        runtime.extend(line[2:])
len(runtime)
result['runtime'] = runtime

In [8]:
result

,#V,#E,Ir,I1mr,ILmr,Dr,D1mr,DLmr,Dw,D1mw,...,flcomps,intops,intcomps,total,cycles,flops/cycle,LL miss rate,mem,branch miss rate,runtime
0,100,500,636472,55,55,96153,692,1,79976,955,...,7819,13649,36298,69922,389294,0.179612,0.006891,176129,2.388168,0.000169991
1,100,500,358516,91,91,89523,169,0,47981,403,...,7819,13649,36298,69922,261295,0.267598,0.019757,137504,13.596059,0.000116511
2,100,500,345193,93,92,90506,281,1,37914,291,...,7819,13649,36298,69922,365875,0.191109,0.019636,128420,13.080375,0.000170921
3,100,500,324962,98,98,92290,215,0,47598,416,...,7819,13649,36298,69922,372043,0.187941,0.023664,139888,14.423225,0.000154351
4,100,500,320406,86,86,67670,135,0,45376,353,...,7819,13649,36298,69922,344799,0.202791,0.019841,113046,16.442348,0.000145031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20000,80000,165903593,92,92,39493517,681613,0,20452829,711066,...,3273707,5586980,14173088,27515528,2482130000,0.011085,0.011948,59946346,12.930325,1.003
80,20000,80000,165566950,78,78,33430932,1840371,0,15693086,267048,...,3273707,5586980,14173088,27515528,243069000,0.113201,0.042358,49124018,17.440600,0.0992371
81,20000,80000,145135690,86,86,29033052,1180207,0,18771000,1045158,...,3273707,5586980,14173088,27515528,213610000,0.128812,0.000113,47804052,15.664714,0.0862049
82,20000,80000,141651351,93,92,34486907,1778623,1,14751004,225901,...,3273707,5586980,14173088,27515528,198732000,0.138456,0.001142,49237911,13.499148,0.0792809


In [9]:
import plotly.express as px
import plotly.graph_objects as go

num_font = dict(size=22,color='#000000')
default_font = dict(family="Helvetica Neue",size=20,color='#000000')
#colors=px.colors.sequential.deep[2:]  # Candidate: Prism
colors=px.colors.sequential.Cividis[::-1][1:]
colorq=px.colors.qualitative.T10
marker_symbols=['circle','square','diamond','cross','triangle-up-open','pentagon','hexagram','star','hourglass', 'asterisk']

In [10]:
fig = go.Figure()
ploted_edge=[100,500,1000,5000,10000, 20000, 40000, 80000]
# for idx, e in enumerate(ploted_edge):
func_alias=['Vector', 'Inline', 'Merge+SIMD', 'Merge', 'Merge+Rand', 'Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[result['function']==func]
    fig.add_trace(go.Scatter(x=same["#V"], y=same['branch miss rate'], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True, legend=dict(traceorder="normal"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("branch-miss-rate.pdf")

In [12]:
fig = go.Figure()
ploted_edge=[100,500,1000,5000,10000, 20000, 40000, 80000]
# for idx, e in enumerate(ploted_edge):
func_alias=['Inline', 'Merge+SIMD', 'Merge+Rand', 'Vector', 'Merge', 'Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func) & (result['#E']==20000)]
    fig.add_trace(go.Scatter(x=same["#V"], y=same['flops/cycle'], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True, legend=dict(traceorder="normal"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("flops_cycles_e20000.pdf")

In [1]:
fig = go.Figure()
ploted_edge=[100,500,1000,5000,10000, 20000, 40000, 80000]
# for idx, e in enumerate(ploted_edge):
func_alias=['Merge+SIMD','Inline','Merge+Rand', 'Merge','Vector','Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func) & (result['#E']==80000)]
    fig.add_trace(go.Scatter(x=same["#V"], y=same['flops/cycle'], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="nonnegative",
                             showline=False),
                  showlegend=True, legend=dict(traceorder="normal"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("flops_cycles_e80000.pdf")

NameError: name 'go' is not defined

In [12]:
fig = go.Figure()
ploted_edge=[100,500,1000,5000,10000, 20000, 40000, 80000]
# for idx, e in enumerate(ploted_edge):
func_alias=['Merge+SIMD','Inline','Merge+Rand', 'Merge','Vector','Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func) & (result['#V']==1000)]
    fig.add_trace(go.Scatter(x=same["#E"], y=same['flops/cycle'], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True, legend=dict(traceorder="normal"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("flops_cycles_v1000.pdf")

In [13]:
fig = go.Figure()
ploted_edge=[100,500,1000,5000,10000, 20000, 40000, 80000]
# for idx, e in enumerate(ploted_edge):
func_alias=['Merge+SIMD','Inline','Merge+Rand', 'Merge','Vector','Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func) & (result['#V']==10000)]
    fig.add_trace(go.Scatter(x=same["#E"], y=same['flops/cycle'], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True, legend=dict(traceorder="normal"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("flops_cycles_v10000.pdf")

In [14]:
# flops/cycle vs oi
fig = go.Figure()
func_alias=['Vector', 'Inline', 'Merge+SIMD', 'Merge', 'Merge+Rand', 'Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func)]
    fig.add_trace(go.Scatter(x=same["flops"]/same["mem"], y=same["flops/cycle"],  name=f"{func}",
                            mode='lines+markers',marker=dict(symbol=marker_symbols[j],size=5),
                            line=dict(color=colorq[j], width=1)))

fig.update_layout(xaxis=dict(showline=True, showgrid=True,
                             linecolor='rgb(0, 0, 0)', linewidth=2,rangemode="nonnegative",
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             rangemode="nonnegative",
                             showline=True),
                  #showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_roofline.pdf")

In [15]:
# runtime v=5000
fig = go.Figure()
func_alias=['Vector', 'Inline', 'Merge+SIMD', 'Merge', 'Merge+Rand', 'Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func)&(result['#E']==10000)]
    fig.add_trace(go.Scatter(x=same['#E'], y=same["runtime"],  name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=True,
                             linecolor='rgb(0, 0, 0)', linewidth=2,rangemode="nonnegative",
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             rangemode="nonnegative",
                             showline=True),
                  #showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("runtime_v5000.pdf")

In [16]:
# runtime v=5000
fig = go.Figure()
func_alias=['Vector', 'Inline', 'Merge+SIMD', 'Merge', 'Merge+Rand', 'Baseline']
for j, func in enumerate(func_alias):
    same = result.loc[(result['function']==func)&(result['#E']==80000)]
    fig.add_trace(go.Scatter(x=same['#V'], y=same["runtime"],  name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colorq[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=True,
                             linecolor='rgb(0, 0, 0)', linewidth=2,rangemode="nonnegative",
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             rangemode="nonnegative",
                             showline=True),
                  #showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("runtime_v5000.pdf")